In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from sklearn.metrics import roc_auc_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
import clip
from pathlib import Path
import shutil

# load in clip
import os
import torch.utils.data as data
import torchvision.transforms as transforms
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
from typing import List, Optional
from PIL import Image

from collections import defaultdict
from tqdm import tqdm

import cv2
import openslide

import cv2
import openslide

In [2]:
clip.__file__

'/opt/conda/envs/pytorch/lib/python3.9/site-packages/clip/__init__.py'

Get Sample Data


In [3]:
data_dir = Path("../../../../data/data/")

In [4]:
# load clip vit models -- this should have pre-trained clip weights
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device, jit=False)

model.load_state_dict(torch.load('./checkpoints/sample_0/checkpoint_63000.pt'))

<All keys matched successfully>

In [5]:
# read in from csv, sample one from each class
labels_path = "slides.csv"
labels_df = pd.read_csv(labels_path)
# # add column to labels 
labels_df["label"] = labels_df.apply(lambda x: x['cases.0.disease_type'].split(" ")[0].lower(),axis=1)
labels_df
sample_label_cases = list(labels_df.groupby("label").first()['cases.0.submitter_id'])

# sample_label_cases
labels_sample = labels_df[labels_df['cases.0.submitter_id'].isin(sample_label_cases)]

In [6]:
labels_sample

,cases.0.disease_type,cases.0.project.project_id,cases.0.samples.0.sample_type,cases.0.samples.1.sample_type,cases.0.submitter_id,file_id,file_name,id,label
0,Squamous Cell Neoplasms,TCGA-CESC,Primary Tumor,NaN,TCGA-MA-AA3X,69037019-9df5-493e-8067-d4078d78e518,TCGA-MA-AA3X-01Z-00-DX1.44657CDB-53F1-4DED-AE5...,69037019-9df5-493e-8067-d4078d78e518,squamous
12,Cystic Mucinous and Serous Neoplasms,TCGA-CESC,Primary Tumor,NaN,TCGA-Q1-A6DV,327f4dc9-5461-44a4-8c29-f16757e5bde1,TCGA-Q1-A6DV-01Z-00-DX1.A31CD86F-703A-462F-860...,327f4dc9-5461-44a4-8c29-f16757e5bde1,cystic
28,Complex Epithelial Neoplasms,TCGA-CESC,Primary Tumor,NaN,TCGA-DS-A7WF,96858e79-abc9-430a-b1be-5c08407a330a,TCGA-DS-A7WF-01A-01-TSA.3E5926D1-9D6F-46F1-A68...,96858e79-abc9-430a-b1be-5c08407a330a,complex
75,Adenomas and Adenocarcinomas,TCGA-CESC,Primary Tumor,NaN,TCGA-C5-A1ME,06da5d60-88a1-460e-924d-7a49d82ab2c0,TCGA-C5-A1ME-01Z-00-DX1.44645529-F452-49AD-804...,06da5d60-88a1-460e-924d-7a49d82ab2c0,adenomas
263,Transitional Cell Papillomas and Carcinomas,TCGA-BLCA,Primary Tumor,NaN,TCGA-G2-A3IB,3aa30334-ac24-477c-a28b-e2cc211748e2,TCGA-G2-A3IB-01A-01-TSA.70AA56B1-1580-4FE2-A83...,3aa30334-ac24-477c-a28b-e2cc211748e2,transitional
732,Mature B-Cell Lymphomas,TCGA-DLBC,Primary Tumor,NaN,TCGA-GS-A9TZ,233e0ac3-e8a6-4132-b4cd-68ddcd3890bd,TCGA-GS-A9TZ-01A-01-TSA.686AE713-E246-41CE-811...,233e0ac3-e8a6-4132-b4cd-68ddcd3890bd,mature
1142,Epithelial Neoplasms NOS,TCGA-BLCA,Primary Tumor,NaN,TCGA-ZF-AA4N,ec9adc5f-a25c-4e89-9c48-58f3bb23523d,TCGA-ZF-AA4N-01A-01-TS1.596D27B2-1418-43B5-8C7...,ec9adc5f-a25c-4e89-9c48-58f3bb23523d,epithelial
1287,Gliomas,TCGA-GBM,Primary Tumor,NaN,TCGA-02-0321,7b4519a1-7d6a-4c47-8563-a044f2448e55,TCGA-02-0321-01Z-00-DX1.3c57c27d-4a4f-45af-bf8...,7b4519a1-7d6a-4c47-8563-a044f2448e55,gliomas
1552,Not Reported,TCGA-GBM,Solid Tissue Normal,NaN,TCGA-06-0675,e412d797-4fce-4595-b5e1-f2448af7dea4,TCGA-06-0675-11A-03-TS3.A650CD6A-E0FE-4C6E-8CC...,e412d797-4fce-4595-b5e1-f2448af7dea4,not
1566,Germ Cell Neoplasms,TCGA-TGCT,Primary Tumor,NaN,TCGA-W4-A7U4,7bf12a3d-6588-4a69-a50b-dd481fa6f3df,TCGA-W4-A7U4-01A-01-TS1.5C5ADDD3-F392-48FD-B83...,7bf12a3d-6588-4a69-a50b-dd481fa6f3df,germ


In [7]:
# Get all labels 
all_cases = list(labels_df["cases.0.submitter_id"])
len(all_cases)

11007

In [8]:
def get_svs_paths(all_svs: List[Path], cases: List[str]):
    # Returns single svs for each patient given a set of cases and a list of all svs
    svs_counts = defaultdict(int)
    svs_paths = []
    for svs_path in all_svs: # iterate through each svs
        for sample_case in cases: # for each case
            # if sample_case in str(svs_path) and "DX" in str(svs_path): 
            if sample_case in str(svs_path):
                if svs_counts[sample_case] == 0: 
                    svs_paths.append(svs_path)
                svs_counts[sample_case] += 1
    return svs_paths

all_svs = list(data_dir.rglob("./slides/*.svs"))
svs_paths = get_svs_paths(all_svs=all_svs, cases=all_cases)



In [9]:
len(all_svs)

10697

Extract Patches


In [10]:
# output_dir = data_dir / "patches"
# extract_patches(paths=svs_paths, output_dir=output_dir)

Inference

Data Loader

In [11]:
class PatchDataset(data.Dataset):
    def __init__(self, patch_paths: List[Path] , labels_df: pd.DataFrame, transform = None):
        """
        Notes:
            assumes `label_df` has columns `cases.0.submitter_id` and `label` mapping
            TCGA id to corresponding class. 
        """
        self.labels_df = labels_df
        self.patch_paths = patch_paths
        self.transform = transform
        
        self.last_idx = 0
        self.labels_to_idx = dict()

    def __len__(self):
        return len(self.patch_paths)

    def __getitem__(self, idx):
        # Get patch and preprocess
        patch_path = self.patch_paths[idx]
        patch_stem = patch_path.stem
        case_id = "-".join(str(patch_stem).split(".")[0].split("-")[:3])
        patch = Image.open(patch_path)
        if self.transform is not None:
            patch = self.transform(patch)
        
        # Get corresponding label
        label = self.labels_df[self.labels_df["cases.0.submitter_id"] == case_id]["label"]
        if len(label) == 0: 
            raise ValueError("Can't find corresponding label a given case.")
        label = label.values[0] # extract single label
        
#         print(label)
        if label not in self.labels_to_idx: 
            self.labels_to_idx[label] = self.last_idx
            self.last_idx += 1

        label_idx = self.labels_to_idx[label]
            
        return patch, label_idx

Load Patches

In [12]:
patch_dir = data_dir / "patches" / "10.0_224"
patch_paths = list(patch_dir.rglob("./*.png"))
print("Number of paths: ", len(patch_paths))
print("Path Director", patch_dir)
# sample_image = preprocess(Image.open(patch_paths[0])).unsqueeze(0).to(device)

Number of paths:  1999969
Path Director ../../../../data/data/patches/10.0_224


In [13]:
unq_ids = np.unique(["-".join(str(patch_path.stem).split(".")[0].split("-")[:3]) for patch_path in patch_paths])
len(unq_ids)

1332

In [14]:
train_ids, test_ids = train_test_split(unq_ids, test_size=0.2, random_state=42)
print(len(train_ids))
print(len(test_ids))

1065
267


In [15]:
# get patches containing each id
def get_patches_from_ids(ids, patch_paths): 
    paths = []
    for id in ids: 
        for patch in patch_paths: 
            if id in str(patch): 
                paths.append(patch)
    return paths

train_paths = get_patches_from_ids(ids=train_ids, patch_paths=patch_paths)
test_paths = get_patches_from_ids(ids=test_ids, patch_paths=patch_paths)


In [16]:
print("Num train paths: ", len(train_paths))

Num train paths:  1606868


In [17]:
print("Num test paths: ", len(test_paths))

Num test paths:  393101


In [34]:
# run model on full test dataset
test_data_dir = patch_dir
# test_dataset = datasets.ImageFolder(test_data_dir, transform=preprocess)
train_labels_df = labels_df[labels_df['cases.0.submitter_id'].isin(train_ids)]
test_labels_df = labels_df[labels_df['cases.0.submitter_id'].isin(test_ids)]


print(train_labels_df['label'].nunique())
print(test_labels_df['label'].nunique())

17
12


In [35]:

train_dataset = PatchDataset(
    patch_paths=train_paths,
    labels_df=train_labels_df, 
    transform=preprocess,
)

test_dataset = PatchDataset(
    patch_paths=test_paths, 
    labels_df=test_labels_df, 
    transform=preprocess,
)

# Create a DataLoader to load the test dataset in batches
batch_size = 32
num_workers = 4
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

Linear Evaluation

First, train a linear model on a train dataset
Second, evaluate linear model on a test datasets

In [24]:
logits_list = []
labels_list = []
torch.multiprocessing.set_sharing_strategy('file_system')
for batch_idx, (images, labels) in tqdm(enumerate(train_loader), total=len(train_loader)):
    with torch.no_grad():
        images = images.to(device)
        features = model.encode_image(images)
        # move to gpu
        features.to(device)
        labels.to(device)
        
    logits_list.append(features)
    labels_list.append(labels)
logits = torch.cat(logits_list, dim=0)
labels = torch.cat(labels_list, dim=0)


100%|██████████| 50215/50215 [38:35<00:00, 21.68it/s] 


In [33]:
logits


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [25]:
# Create a DataLoader to load the test dataset in batches
num_ftrs = 512
num_classes = train_labels_df['label'].nunique()

# Define the linear classifier and the loss function
linear_classifier = nn.Linear(num_ftrs, num_classes)
linear_classifier = linear_classifier.to(device)
criterion = nn.CrossEntropyLoss()

# Define the optimizer for the linear classifier
optimizer = optim.SGD(linear_classifier.parameters(), lr=0.01, momentum=0.9)

# Set the linear classifier to training mode
linear_classifier.train()

Linear(in_features=512, out_features=4, bias=True)

In [30]:
# Train the linear classifier on the pre-trained logits using the test dataset
num_epochs = 30
for epoch in tqdm(range(num_epochs)):
    running_loss = 0.0
    # train for each of the logits and labels, want to batch this if possible
    for i, (features, targets) in enumerate(zip(logits, labels)):
        optimizer.zero_grad()

        features = features.to(device)
        targets = targets.to(device)


        outputs = linear_classifier(features.float())
        loss = criterion(outputs.unsqueeze(0), targets.unsqueeze(0))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print('Epoch [%d/%d], Loss: %.4f' % (epoch+1, num_epochs, running_loss))

  0%|          | 0/30 [00:07<?, ?it/s]


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:
# # logits_list = []
# # labels_list = []

# # TODO: move into a function
# num_epochs = 30
# for epoch in range(num_epochs):
#     running_loss = 0.0
#     for batch_idx, (images, labels) in tqdm(enumerate(train_loader), total=len(train_loader)):
#         with torch.no_grad():
#             images = images.to(device)
#             features = model.encode_image(images)
#             features.to(device)
#             labels_one_hot = torch.nn.functional.one_hot(labels, num_classes=4).float().to(device) # convert labels to one-hot encoding
#             outputs = linear_classifier(features.float())
#             loss = criterion(outputs, labels_one_hot)
#             loss.backward()
#             optimizer.step()
#             running_loss += loss.item()
    
#     print('Epoch [%d/%d], Loss: %.4f' % (epoch+1, num_epochs, running_loss))

# # logits = torch.cat(logits_list, dim=0)
# # labels = torch.cat(labels_list, dim=0)

In [ ]:
# compute the model predictions for the test dataset
y_pred = []
y_true = []
model.eval()
with torch.no_grad():
    for images, labels in tqdm(test_loader):
        images = images.to(device)
        labels = labels.to(device)

        output = model.encode_image(images) # clip 
        output = linear_classifier(output.float())
        
        y_pred.append(output)
        y_true.append(labels)

In [ ]:
# assume y_pred is a tensor of shape (num_examples, num_classes) containing the outputs of the linear classifier
# and y_true is a tensor of shape (num_examples) containing the categorical labels
y_pred = torch.cat(y_pred, dim=0)
y_true = torch.cat(y_true, dim=0)

y_prob = torch.softmax(y_pred, dim=1)
y_true = y_true.detach().cpu().numpy()
y_prob = y_prob.detach().cpu().numpy()
# apply softmax to the outputs of the linear classifier to obtain the predicted probabilities

# compute the AUROC for each class using the predicted probabilities and the corresponding categorical labels
aurocs = []
for c in range(num_classes):
    y_true_c = (y_true == c)
    y_prob_c = y_prob[:, c]
    auroc_c = roc_auc_score(y_true_c, y_prob_c)
    aurocs.append(auroc_c)

# compute the mean AUROC over all classes
mean_auroc = sum(aurocs) / num_classes

In [ ]:
print("multiclass AUROC: ", aurocs, mean_auroc)

In [ ]:
# acc@k, prec@k, rec@k
def top_k_acc_prec_rec(y_prob, y_true, k):
    # y_prob is a numpy array of shape (num_examples, num_classes) containing the predicted probabilities
    # y_true is a numpy array of shape (num_examples) containing the true labels
    # k is the number of classes to consider for top-k accuracy, precision, and recall

    # get the indices of the top k predicted probabilities for each example
    top_k_preds = np.argsort(y_prob, axis=1)[:, -k:]

    # compute accuracy@k, precision@k, and recall@k for each example
    acc_k = np.mean(np.any(top_k_preds == y_true[:, np.newaxis], axis=1))
    prec_k = np.mean([np.any(top_k_preds[i] == y_true[i]) for i in range(len(y_true))])
    rec_k = np.mean([np.any(y_true[i] == top_k_preds[i]) for i in range(len(y_true))])

    return acc_k, prec_k, rec_k

In [ ]:
acc_k, prec_k, rec_k = top_k_acc_prec_rec(y_prob, y_true, k=2)

In [ ]:
print("acc@k: ", acc_k)
print("prec@k: ", prec_k)
print("rec@k: ", rec_k)

In [ ]:
print(mean_auroc)